## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [1]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['/home/nirojasvar/random_sast/ExperimentationRSAST',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python310.zip',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/lib-dynload',
 '',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nirojasvar/random_sast/cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/cd_diagram']

In [2]:
from sast import *
import pandas as pd
import researchpy
import math
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [3]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [4]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'SAST', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [5]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

directories=[]
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST')
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_All')
#directories.append(os.getcwd()+'/ResultsRsast/Server16_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/results_accuracy_per_ds')
#directories.append(os.getcwd()+'/results_accuracy_per_ds_10000')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


/tmp/ipykernel_9975/1017712830.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_9975/1017712830.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_9975/1017712830.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_9975/1017712830.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_9975/1017712830.py:29: FutureWarning: The default value of rege

In [6]:
df_used_sast=pd.read_excel("DataSetsUCLASummary.xlsx", dtype=str)
df_used_sast.columns

Index(['USED SAST', 'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'ID', 'Name', 'Type', 'Train ', 'Test ', 'Class', 'Length',
       'RSAST (10,10) WORST SAST?', 'O(|c|nm²)', 'O(|c|nm³)',
       'Approx Time (hours)', 'L25', 'L50', 'L75', 'ED (w=0)',
       'DTW (learned_w) ', 'DTW (w=100)', 'Default rate', 'Data donor/editor',
       '50/L', 'Unnamed: 24'],
      dtype='object')

In [7]:
df_used_sast=df_used_sast[['Name', 'Type', 'Train ', 'Test ', 'Class', 'Length','USED SAST']]
df_used_sast=df_used_sast[df_used_sast['USED SAST']=='Y']
df_used_sast

,Name,Type,Train,Test,Class,Length,USED SAST
0,SmoothSubspace,Simulated,150,150,3,15,Y
1,Chinatown,Traffic,20,343,2,24,Y
2,ItalyPowerDemand,Sensor,67,1029,2,24,Y
3,MelbournePedestrian,Traffic,1194,2439,10,24,Y
4,Crop,Image,7200,16800,24,46,Y
...,...,...,...,...,...,...,...
77,BeetleFly,Image,20,20,2,512,Y
78,BirdChicken,Image,20,20,2,512,Y
79,Herring,Image,64,64,2,512,Y
80,Earthquakes,Sensor,322,139,2,512,Y


In [8]:
"""
len_methods=['ACF&PACF: n_random_points=30 nb_inst_per_class=1',
'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
'None: n_random_points=30 nb_inst_per_class=1',
'None: n_random_points=10 nb_inst_per_class=10'
]
df_result=df_result[df_result.classifier_name.isin(len_methods)]
"""


"\nlen_methods=['ACF&PACF: n_random_points=30 nb_inst_per_class=1',\n'ACF&PACF: n_random_points=10 nb_inst_per_class=10',\n'None: n_random_points=30 nb_inst_per_class=1',\n'None: n_random_points=10 nb_inst_per_class=10'\n]\ndf_result=df_result[df_result.classifier_name.isin(len_methods)]\n"

In [9]:
#get ds tested overall
print("Total DS:" + str(len(df_result['dataset_name'].unique())))

Total DS:16


In [10]:
#get ds tested overall
df_result['dataset_name'].unique()

array(['MiddlePhalanxOutlineAgeGroup', 'DistalPhalanxOutlineAgeGroup',
       'ItalyPowerDemand', 'ProximalPhalanxOutlineCorrect',
       'SonyAIBORobotSurface2', 'ProximalPhalanxOutlineAgeGroup',
       'MiddlePhalanxOutlineCorrect', 'ProximalPhalanxTW',
       'SonyAIBORobotSurface1', 'SmoothSubspace', 'Chinatown',
       'DistalPhalanxOutlineCorrect', 'MiddlePhalanxTW', 'Crop',
       'SyntheticControl', 'DistalPhalanxTW'], dtype=object)

In [11]:
#get ds tested overall
print("Total Param Combination:" + str(len(df_result['classifier_name'].unique())))

Total Param Combination:4


In [12]:
#get len methods of generated datasets
df_result.classifier_name.unique()

array(['all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [13]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

In [14]:
pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index()

classifier_name,dataset_name,all: n_random_points=10 nb_inst_per_class=1,all: n_random_points=10 nb_inst_per_class=10,all: n_random_points=30 nb_inst_per_class=1,all: n_random_points=30 nb_inst_per_class=10
0,Chinatown,0.971429,0.985423,0.977259,0.985423
1,Crop,0.634568,0.744851,0.701875,0.750417
2,DistalPhalanxOutlineAgeGroup,0.735252,0.722302,0.723741,0.715108
3,DistalPhalanxOutlineCorrect,0.727536,0.730435,0.701449,0.724638
4,DistalPhalanxTW,0.621583,0.627338,0.617266,0.620144
5,ItalyPowerDemand,0.952187,0.954908,0.956074,0.953741
6,MiddlePhalanxOutlineAgeGroup,0.546753,0.518182,0.494805,0.523377
7,MiddlePhalanxOutlineCorrect,0.813058,0.781443,0.813746,0.778007
8,MiddlePhalanxTW,0.506494,0.501299,0.496104,0.497403
9,ProximalPhalanxOutlineAgeGroup,0.834146,0.790244,0.821463,0.788293


In [15]:
"""
df_tunned_rsast=df_used_sast.merge(pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index(),how='right',right_on="dataset_name",left_on="Name").round(2)[['Name', 'ACF&PACF: n_random_points=30 nb_inst_per_class=10','ACF&PACF: n_random_points=30 nb_inst_per_class=10','ACF&PACF: n_random_points=30 nb_inst_per_class=1','Type', 'Train ', 'Test ', 'Class', 'Length']]
df_tunned_rsast.sort_values("Name",inplace=True)
df_tunned_rsast.to_excel('ds_runned_rsast.xlsx')
"""

'\ndf_tunned_rsast=df_used_sast.merge(pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index(),how=\'right\',right_on="dataset_name",left_on="Name").round(2)[[\'Name\', \'ACF&PACF: n_random_points=30 nb_inst_per_class=10\',\'ACF&PACF: n_random_points=30 nb_inst_per_class=10\',\'ACF&PACF: n_random_points=30 nb_inst_per_class=1\',\'Type\', \'Train \', \'Test \', \'Class\', \'Length\']]\ndf_tunned_rsast.sort_values("Name",inplace=True)\ndf_tunned_rsast.to_excel(\'ds_runned_rsast.xlsx\')\n'

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [16]:
filter_param=df_result

#filter_param=filter_param[filter_param.len_method=='ACF&PACF']
filter_param.dataset_name.unique()

array(['MiddlePhalanxOutlineAgeGroup', 'DistalPhalanxOutlineAgeGroup',
       'ItalyPowerDemand', 'ProximalPhalanxOutlineCorrect',
       'SonyAIBORobotSurface2', 'ProximalPhalanxOutlineAgeGroup',
       'MiddlePhalanxOutlineCorrect', 'ProximalPhalanxTW',
       'SonyAIBORobotSurface1', 'SmoothSubspace', 'Chinatown',
       'DistalPhalanxOutlineCorrect', 'MiddlePhalanxTW', 'Crop',
       'SyntheticControl', 'DistalPhalanxTW'], dtype=object)

In [17]:
filter_param.rpoint.unique()

array(['10', '30'], dtype=object)

In [18]:
#filter_param=filter_param[filter_param.rpoint=='half_len']

In [19]:
filter_param.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [20]:
#filter_param=filter_param[filter_param.nb_per_class=='30']

In [21]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="SAST"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df


,dataset_name,all,score,method
0,Chinatown,0.979883,0.96,SAST
1,Crop,0.707928,0.73,SAST
2,DistalPhalanxOutlineAgeGroup,0.724101,0.76,SAST
3,DistalPhalanxOutlineCorrect,0.721014,0.74,SAST
4,DistalPhalanxTW,0.621583,0.67,SAST
5,ItalyPowerDemand,0.954227,0.96,SAST
6,MiddlePhalanxOutlineAgeGroup,0.520779,0.53,SAST
7,MiddlePhalanxOutlineCorrect,0.796564,0.83,SAST
8,MiddlePhalanxTW,0.500325,0.56,SAST
9,ProximalPhalanxOutlineAgeGroup,0.808537,0.85,SAST


In [22]:
merged_df.describe()

,all,score
count,16.000000,16.00000
mean,0.777397,0.79625
std,0.145255,0.13311
min,0.500325,0.53000
25%,0.717743,0.73750
50%,0.784257,0.80500
75%,0.873677,0.88000
max,0.980333,0.98000


In [23]:

# Summary statistics for a Series (single variable)

summ_ds=filter_param
summ_ds=researchpy.summary_cont(summ_ds.groupby(['dataset_name'])['accuracy'], conf = 0.95)
summ_ds

,N,Mean,SD,SE,95% Conf.,Interval
dataset_name,,,,,,
Chinatown,20,0.9799,0.0081,0.0018,0.9761,0.9837
Crop,16,0.7079,0.0479,0.0120,0.6824,0.7334
DistalPhalanxOutlineAgeGroup,20,0.7241,0.0210,0.0047,0.7143,0.7339
DistalPhalanxOutlineCorrect,20,0.7210,0.0176,0.0039,0.7128,0.7292
DistalPhalanxTW,20,0.6216,0.0181,0.0041,0.6131,0.6301
ItalyPowerDemand,20,0.9542,0.0058,0.0013,0.9515,0.9569
MiddlePhalanxOutlineAgeGroup,20,0.5208,0.0248,0.0055,0.5092,0.5324
MiddlePhalanxOutlineCorrect,20,0.7966,0.0216,0.0048,0.7865,0.8067
MiddlePhalanxTW,20,0.5003,0.0181,0.0040,0.4919,0.5088


In [24]:
stats = filter_param.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std  ci95_hi   ci95_lo
len_method                                              
all         0.778277    316  0.143876  0.79414  0.762413


#### Generate Boxplot Tunning Hyperparameter

In [25]:
df_result.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [26]:
df_result.len_method.unique()

array(['all'], dtype=object)

In [27]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_result.nb_per_class.unique()):

    for len_m in df_result.len_method.unique():
        
        
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.accuracy)
        min_bx=min(df_to_cd.accuracy)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_acc/boxplot_acc'+len_m+ints+'.png')




In [28]:
df_result.classifier_name.unique()

array(['all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [29]:
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
            print("ds:"+ds)
            print("ints:"+str(ints))
            print("len_m:"+len_m)

            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.accuracy)
            min_bx=min(df_to_cd.accuracy)
            
            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]    

            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            
            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_acc_per_ds/boxplot_acc'+len_m+ints+'_'+ds+'.png')

ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:all
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:all
ds:DistalPhalanxOutlineAgeGroup
ints:1
len_m:all
ds:DistalPhalanxOutlineAgeGroup
ints:10
len_m:all
ds:ItalyPowerDemand
ints:1
len_m:all
ds:ItalyPowerDemand
ints:10
len_m:all
ds:ProximalPhalanxOutlineCorrect
ints:1
len_m:all
ds:ProximalPhalanxOutlineCorrect
ints:10
len_m:all
ds:SonyAIBORobotSurface2
ints:1
len_m:all
ds:SonyAIBORobotSurface2
ints:10
len_m:all
ds:ProximalPhalanxOutlineAgeGroup
ints:1
len_m:all
ds:ProximalPhalanxOutlineAgeGroup
ints:10
len_m:all
ds:MiddlePhalanxOutlineCorrect
ints:1
len_m:all
ds:MiddlePhalanxOutlineCorrect
ints:10
len_m:all
ds:ProximalPhalanxTW
ints:1
len_m:all
ds:ProximalPhalanxTW
ints:10
len_m:all
ds:SonyAIBORobotSurface1
ints:1
len_m:all
ds:SonyAIBORobotSurface1
ints:10
len_m:all
ds:SmoothSubspace
ints:1
len_m:all


/tmp/ipykernel_9975/3028681140.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


ds:SmoothSubspace
ints:10
len_m:all
ds:Chinatown
ints:1
len_m:all
ds:Chinatown
ints:10
len_m:all
ds:DistalPhalanxOutlineCorrect
ints:1
len_m:all
ds:DistalPhalanxOutlineCorrect
ints:10
len_m:all
ds:MiddlePhalanxTW
ints:1
len_m:all
ds:MiddlePhalanxTW
ints:10
len_m:all
ds:Crop
ints:1
len_m:all
ds:Crop
ints:10
len_m:all
ds:SyntheticControl
ints:1
len_m:all
ds:SyntheticControl
ints:10
len_m:all
ds:DistalPhalanxTW
ints:1
len_m:all
ds:DistalPhalanxTW
ints:10
len_m:all


In [30]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_result.nb_per_class.unique()):
    for len_m in df_result.len_method.unique():
        
        
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
        df_to_cd["time"]=df_to_cd["time"]/60
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.time)
        min_bx=min(df_to_cd.time)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_time/boxplot_time'+len_m+ints+'.png')




/tmp/ipykernel_9975/4020300904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
/tmp/ipykernel_9975/4020300904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd["time"]/60
/tmp/ipykernel_9975/4020300904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [31]:
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
        


            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.time)
            min_bx=min(df_to_cd.time)

            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]


            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            

            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_time_per_ds/boxplot_time'+len_m+ints+'_'+ds+'.png')


#### Generate CD Diagram Tunning Hyperparameter

In [32]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_perf.classifier_name.unique()

array(['all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [33]:
len_m="None"
ints="10"
df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')].classifier_name.unique()

array([], dtype=object)

In [34]:


from cd_function import *
#generate hyperparameter tuning cd diagrams

best_comb_by_method=[]
best=""
for len_m in df_result.len_method.unique():
   df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')]
   draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison", fname='images_cd_diagram/cd-diagram_'+len_m+'.png')
   _, average_ranks, _ = wilcoxon_holm(df_perf=df_to_cd)
   min_rank= min(average_ranks)
   average_ranks=pd.DataFrame(average_ranks)
   best=average_ranks[average_ranks[0]==min_rank][0].index
   best_comb_by_method.append(best[0])



the null hypothesis over the entire classifiers cannot be rejected


NameError: name 'exit' is not defined

In [35]:
type(best_comb_by_method)

list

In [36]:

df_to_cd=df_perf[df_result.classifier_name.isin(best_comb_by_method)]
draw_cd_diagram(df_to_cd, labels=True, title="Best combination comparison", fname='images_cd_diagram/cd-diagram_best_com.png')


/tmp/ipykernel_9975/2209284884.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_to_cd=df_perf[df_result.classifier_name.isin(best_comb_by_method)]


ValueError: At least 3 sets of samples must be given for Friedman test, got 0.

In [ ]:
draw_cd_diagram(df_perf, labels=True, fname='images_cd_diagram/cd-diagram_all.png')


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of th

{0: 'all: n_random_points=10 nb_inst_per_class=1', 1: 'all: n_random_points=30 nb_inst_per_class=1', 2: 'all: n_random_points=10 nb_inst_per_class=10', 3: 'all: n_random_points=30 nb_inst_per_class=10'}
[0, 1, 2, 3]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial


#### Focus on most accurate lenght method

##### Overall Accuracy

In [37]:
# highest accurate hyperparameters
best_comb_by_method

[]

In [38]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result
#df_result_acc=df_result_acc[df_result_acc["dataset_name"]=="Fungi"]

pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.773358,0.783783,NaN,NaN,NaN,NaN,NaN
30,NaN,0.772399,0.783567,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# create a pivot table with the variatioon of score by hyperparameter
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.020639,0.019543,NaN,NaN,NaN,NaN,NaN
30,NaN,0.022631,0.019377,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Overall time complexity

In [ ]:
df_result_acc.time.sum()/(60*60*24)

6.857479894351628

In [ ]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='sum')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,time
dataset_name,,,,,
Adiac,0.356932,6.251375,867.444809,5.253082,879.326493
ArrowHead,0.068162,2.550791,14.467688,0.137025,17.226493
BME,0.037609,0.291632,3.115769,0.036141,3.482713
Beef,0.141614,36.995622,58.747155,0.014530,95.910451
BeetleFly,0.094815,33.239842,16.043617,0.007177,49.390765
...,...,...,...,...,...
TwoPatterns,0.093418,0.414763,118.326124,4.722428,123.560247
UMD,0.043190,0.443225,5.108370,0.119882,5.716431
Wafer,0.082004,0.327180,50.331497,1.112268,51.854631


In [ ]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.311962,2.305446,NaN,NaN,NaN,NaN,NaN
30,NaN,0.801771,6.261969,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Calculate weights time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,0.107884,0.103960
30,0.104333,0.104326


##### Finding subsequences time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,1.72407,14.828489
30,1.81173,14.916789


##### Transform Dataset time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,14.348977,120.052097
30,42.526258,355.997983


##### Classifier time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,2.532497,3.337764
30,3.660759,4.690928
